# Distributed Tracing Template

 Illustrate the configuration for allowing distributed tracing using Jaeger.
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)


# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

## Install Jaeger

We will use the Jaeger All-in-1 resource found at the [Jaeger Kubernetes repo](https://github.com/jaegertracing/jaeger-kubernetes).

In [ ]:
!kubectl create -f https://raw.githubusercontent.com/jaegertracing/jaeger-kubernetes/master/all-in-one/jaeger-all-in-one-template.yml -n seldon

### Start Jaeger UI

```
minikube service jaeger-query -n seldon
```

## Install Seldon

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true --namespace seldon
!helm install ../../../helm-charts/seldon-core --name seldon-core --namespace seldon

## Create Jaeger ConfigMap

In [ ]:
!pygmentize tracing-configmap.yaml

In [ ]:
!kubectl apply -f tracing-configmap.yaml -n seldon

## Run Example REST Deployment

In [ ]:
!pygmentize deployment_rest.json

In [ ]:
!kubectl create -f deployment_rest.json

In [ ]:
!kubectl get sdep tracing-example -o jsonpath='{.status}' 

In [ ]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

Check the Jaeger UI. You should be able to find traces like below:

<img src="jaeger-ui-rest-example.png"/>

In [ ]:
!kubectl delete -f deployment_rest.json

## Run Example GRPC Deployment

In [ ]:
!kubectl create -f deployment_grpc.json

In [ ]:
!kubectl get sdep tracing-example -o jsonpath='{.status}' 

In [ ]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --grpc \
    --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

Check the Jaeger UI. You should be able to find traces like below:

<img src="jaeger-ui-grpc-example.png"/>

In [ ]:
!kubectl delete -f deployment_grpc.json